https://colab.research.google.com/drive/1VoYNfYDKcKRQRor98Zbf2-9VQTtGJ24k?usp=sharing#scrollTo=FuXIFTFapAMI

https://huggingface.co/blog/falcon

In [1]:
import torch
#%pip install transformers
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM 

/home/skynet3/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#!pip install -q -U bitsandbytes
#!pip install -q -U git+https://github.com/huggingface/transformers.git 
#!pip install -q -U git+https://github.com/huggingface/peft.git
#!pip install -q -U git+https://github.com/huggingface/accelerate.git
#!pip install -q datasets
#!pip install scipy
#!pip install einops
!pip install triton


In [2]:
from transformers import AutoTokenizer
import transformers
import torch

model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_new_tokens=10
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.80s/it]
A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'
The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCaus

https://huggingface.co/databricks/dolly-v2-12b/discussions/2
NameError: name 'init_empty_weights' is not defined
"This means you didn't install accelerate. See https://github.com/databrickslabs/dolly for more notes on generation"

 UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication

In [3]:
print(pipeline(["Answer the following question:\nQ.What is the capital for Italy?\nA."]))

/home/skynet3/anaconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


[[{'generated_text': 'Answer the following question:\nQ.What is the capital for Italy?\nA.Rome'}]]


Increasing max new tokens to 500 makes this incredibly slow. It answers the question but then it starts a new line and just keeps rambling the same answer phrased differently over and over.

In [10]:
question= "Answer the following question:\nQ.What is the capital for Texas?\nA."
answer=pipeline([question])[0][0]['generated_text']
print(answer.replace(question,''))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Austin


In [64]:
narrative="The Cuban Missile Crisis, also known as the October Crisis (of 1962) (Spanish: Crisis de Octubre) in Cuba, the Caribbean Crisis (Russian: Карибский кризис, tr. Karibsky krizis, IPA: [kɐˈrʲipskʲɪj ˈkrʲizʲɪs]) in Russia, or the Missile Scare, was a 13-day (October 16 – October 29, 1962) confrontation between the United States and the Soviet Union, which escalated into an international crisis when American deployments of missiles in Italy and Turkey were matched by Soviet deployments of similar ballistic missiles in Cuba. Despite the short time frame, the Cuban Missile Crisis remains a defining moment in American national security and nuclear war preparation. The confrontation is often considered the closest the Cold War came to escalating into a full-scale conflict, nuclear war.[4]"
question= "Which countries are mentioned in the text? Do not exclude any countries explicitly mentioned and do not include any countries not explicitly mentioned."
prompt="Text\n" + narrative + "\nQuestion\n" + question + "\nAnswer\n"
answer=pipeline([prompt])[0][0]['generated_text']


/home/skynet3/anaconda3/lib/python3.10/site-packages/transformers/pipelines/base.py:1079: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [65]:
print([answer.replace(prompt,'')])

['The text mentions the United States, the Soviet Union, Cuba, Turkey, and Italy.']


In [66]:
narrative="The Cuban Missile Crisis, also known as the October Crisis (of 1962) (Spanish: Crisis de Octubre) in Cuba, the Caribbean Crisis (Russian: Карибский кризис, tr. Karibsky krizis, IPA: [kɐˈrʲipskʲɪj ˈkrʲizʲɪs]) in Russia, or the Missile Scare, was a 13-day (October 16 – October 29, 1962) confrontation between the United States and the Soviet Union, which escalated into an international crisis when American deployments of missiles in Italy and Turkey were matched by Soviet deployments of similar ballistic missiles in Cuba. Despite the short time frame, the Cuban Missile Crisis remains a defining moment in American national security and nuclear war preparation. The confrontation is often considered the closest the Cold War came to escalating into a full-scale conflict, nuclear war.[4]"
question= "Which dates are mentioned in the text? Do not exclude any dates explicitly mentioned and do not include any dates not explicitly mentioned."
prompt="Text\n" + narrative + "\nQuestion\n" + question + "\nAnswer\n"
answer=pipeline([prompt])[0][0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [67]:
print([answer.replace(prompt,'')])

['- October 16, 1962\n- October 29, 1962']


In [68]:
narrative="The Cuban Missile Crisis, also known as the October Crisis (of 1962) (Spanish: Crisis de Octubre) in Cuba, the Caribbean Crisis (Russian: Карибский кризис, tr. Karibsky krizis, IPA: [kɐˈrʲipskʲɪj ˈkrʲizʲɪs]) in Russia, or the Missile Scare, was a 13-day (October 16 – October 29, 1962) confrontation between the United States and the Soviet Union, which escalated into an international crisis when American deployments of missiles in Italy and Turkey were matched by Soviet deployments of similar ballistic missiles in Cuba. Despite the short time frame, the Cuban Missile Crisis remains a defining moment in American national security and nuclear war preparation. The confrontation is often considered the closest the Cold War came to escalating into a full-scale conflict, nuclear war.[4]"
question= "What is the final sentence of the text? Do not exclude any parts of the final sentence and do not include any sentences before the last."
prompt="Text\n" + narrative + "\nQuestion\n" + question + "\nAnswer\n"
answer=pipeline([prompt])[0][0]['generated_text']

/home/skynet3/anaconda3/lib/python3.10/site-packages/transformers/pipelines/base.py:1079: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [69]:
print([answer.replace(prompt,'')])

['The final sentence of the text is "The Cuban Missile Crisis ended on October 29, 1962, when the Soviets agreed to remove their missiles from Cuba in exchange for the withdrawal of American missiles from Turkey and Italy."']


In [70]:
narrative="The Cuban Missile Crisis, also known as the October Crisis (of 1962) (Spanish: Crisis de Octubre) in Cuba, the Caribbean Crisis (Russian: Карибский кризис, tr. Karibsky krizis, IPA: [kɐˈrʲipskʲɪj ˈkrʲizʲɪs]) in Russia, or the Missile Scare, was a 13-day (October 16 – October 29, 1962) confrontation between the United States and the Soviet Union, which escalated into an international crisis when American deployments of missiles in Italy and Turkey were matched by Soviet deployments of similar ballistic missiles in Cuba. Despite the short time frame, the Cuban Missile Crisis remains a defining moment in American national security and nuclear war preparation. The confrontation is often considered the closest the Cold War came to escalating into a full-scale conflict, nuclear war.[4]"
question= "What events does the text say took place between the Soviet Union and Turkey?"
prompt="Text\n" + narrative + "\nQuestion\n" + question + "\nAnswer\n"
answer=pipeline([prompt])[0][0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [71]:
print([answer.replace(prompt,'')])

['The text does not provide any specific events that took place between the Soviet Union and Turkey.']


In [72]:
narrative="The Cuban Missile Crisis, also known as the October Crisis (of 1962) (Spanish: Crisis de Octubre) in Cuba, the Caribbean Crisis (Russian: Карибский кризис, tr. Karibsky krizis, IPA: [kɐˈrʲipskʲɪj ˈkrʲizʲɪs]) in Russia, or the Missile Scare, was a 13-day (October 16 – October 29, 1962) confrontation between the United States and the Soviet Union, which escalated into an international crisis when American deployments of missiles in Italy and Turkey were matched by Soviet deployments of similar ballistic missiles in Cuba. Despite the short time frame, the Cuban Missile Crisis remains a defining moment in American national security and nuclear war preparation. The confrontation is often considered the closest the Cold War came to escalating into a full-scale conflict, nuclear war.[4]"
question= "What events does the text say took place between the Soviet Union and Cuba?"
prompt="Text\n" + narrative + "\nQuestion\n" + question + "\nAnswer\n"
answer=pipeline([prompt])[0][0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [73]:
print([answer.replace(prompt,'')])

['The Cuban Missile Crisis took place between the Soviet Union and Cuba.']


In [78]:
from transformers import AutoTokenizer
import transformers
import torch

model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_new_tokens=1 #A single yes/no model
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.95s/it]
The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHe

In [79]:
prompt="The story so far:\n\n Question: Does the next sentence of the story below describe a unilateral action? \n Bolshevik Russia, precursor to the Soviet Union \n Answer:"
answer=pipeline([prompt])[0][0]['generated_text']

/home/skynet3/anaconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [80]:
print([answer.replace(prompt,'')])

[' Yes']


In [81]:
prompt="The story so far:\n Bolshevik Russia, precursor to the Soviet Union \n Question: Does the next sentence of the story below describe a unilateral action? \n The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918. \n Answer:"
answer=pipeline([prompt])[0][0]['generated_text']


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [83]:
print([answer.replace(prompt,'')])

[' Yes']


In [114]:
prompt="The story so far: 'Bolshevik Russia, precursor to the Soviet Union The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918.' \n Question: Does the next sentence of the story describe a unilateral action? \n The next sentence of the story: 'Russia and Germany signed a treaty.' \n Answer:"
answer=pipeline([prompt])[0][0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [115]:
print([answer.replace(prompt,'')])

[' Yes']


In [8]:
coding_instructions = "Does the sentence describe an action or an interaction? Action describes an action, not an interaction, meaning that only one actor is taking an action. Interaction occurs in tandem with another actor and/or involve another actor."
sentence="They then signed a different treaty in 1980."
prompt=coding_instructions + "\n" + sentence + "\nA:"
answer=pipeline([prompt])[0][0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [9]:
print([answer])

['Does the sentence describe an action or an interaction? Action describes an action, not an interaction, meaning that only one actor is taking an action. Interaction occurs in tandem with another actor and/or involve another actor.\nThey then signed a different treaty in 1980.\nA: They signed a different treaty in 1980.']
